# **W1D1: Introduction to the Climate System and Xarray**

## Overview

The first three tutorials of today will introduce the basics of gridded, labeled data with [Xarray](https://xarray.dev/). Since Xarray introduces additional abstractions on top of plain arrays of data, our goal is to show why these abstractions are useful and how they frequently lead to simpler, more robust code.

We'll cover these topics:

1. Create a `DataArray`, one of the core object types in Xarray
1. Understand how to use named coordinates and metadata in a `DataArray`
1. Combine individual `DataArrays` into a `Dataset`, the other core object type in Xarray
1. Subset, slice, and interpolate the data using named coordinates
1. Open netCDF data using XArray
1. Basic subsetting and aggregation of a `Dataset`
1. Brief introduction to plotting with Xarray

# **Tutorial 1: Creating DataArrays and Datasets to Assess Global Climate Data**


**Week 1, Day 1, Introduction to the Climate System**

**Content creators:** Sloane Garelick, Julia Kent

**Content reviewers:** DKatrina Dobson, Younkap Nina Duplex, Danika Gupta, Danika Gupta, Maria Gonzalez, Will Gregory, Nahid Hasan, Sherry Mi, Beatriz Cosenza Muralles 

**Content editors:** Yosmely Bermúdez 

**Production editors:** Wesley Banfield, Jenna Pearson, Chi Zhang, Ohad Zivan

**Our 2023 Sponsors:** NASA TOPS

### **Code and Data Sources**

Code and data for this tutorial is based on existing content from [Project Pythia](https://foundations.projectpythia.org/core/xarray/xarray-intro.html).

# **Tutorial Objectives**


As you just learned in the Introduction to Climate video, variations in global climate involve various forcings, feedbacks, and interactions between multiple processes and systems. Because of this complexity, global climate datasets are often very large with multiple dimensions and variables.

One useful computational tool for organizing, analyzing and interpreting large global datasets is XArray, an open source project and Python package that makes working with labelled multi-dimensional arrays simple and efficient.

In this first tutorial, we will use the `DataArray` and `Dataset` objects, which are used to represent and manipulate spatial data, to practice organizing large global climate datasets and to understand variations in Earth's climate system.

In [2]:
# @title Video 1: Speaker Introduction
#Tech team will add code to format and display the video

# Setup

Simmilar to `numpy`, `np`; `pandas`, `pd`; you may often encounter `xarray` imported within a shortened namespace as `xr`. `pythia_datasets` provides example data for us to work with.

In [1]:
!pip install datetime

!pip install numpy
!pip install pandas
!pip install xarray
!pip install pythia_datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 kB 12.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.4/202.4 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 13.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 12.6 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.3/994.3 kB 12.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.8 MB/s eta 0:00:00


In [2]:
from datetime import timedelta

import numpy as np
import pandas as pd
import xarray as xr
from pythia_datasets import DATASETS

In [5]:
# @title Figure Settings
import ipywidgets as widgets       # interactive display
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/ClimateMatchAcademy/course-content/main/cma.mplstyle")

ModuleNotFoundError: No module named 'ipywidgets'

# Introducing the `DataArray` and `Dataset`

Xarray expands on the capabilities on NumPy arrays, providing a lot of streamlined data manipulation. It is similar in that respect to Pandas, but whereas Pandas excels at working with tabular data, Xarray is focused on N-dimensional arrays of data (i.e. grids). Its interface is based largely on the netCDF data model (variables, attributes, and dimensions), but it goes beyond the traditional netCDF interfaces to provide functionality similar to netCDF-java's [Common Data Model (CDM)](https://docs.unidata.ucar.edu/netcdf-java/current/userguide/common_data_model_overview.html). 

# Section 1: Creation of a `DataArray` object

The `DataArray` is one of the basic building blocks of Xarray (see docs [here](http://xarray.pydata.org/en/stable/user-guide/data-structures.html#dataarray)). It provides a `numpy.ndarray`-like object that expands to provide two critical pieces of functionality:

1. Coordinate names and values are stored with the data, making slicing and indexing much more powerful
2. It has a built-in container for attributes

Here we'll initialize a `DataArray` object by wrapping a plain NumPy array, and explore a few of its properties.

## Section 1.1: Generate a random numpy array

For our first example, we'll just create a random array of "temperature" data in units of Kelvin:

In [30]:
data = 283 + 5 * np.random.randn(5, 3, 4)
data

array([[[281.62864287, 286.77948586, 284.45803386, 286.67998675],
        [281.57324961, 275.35379738, 285.34217787, 293.58498361],
        [291.04613942, 279.4760685 , 283.5393635 , 285.77503107]],

       [[290.81933549, 287.79205845, 290.07149916, 283.62520231],
        [282.87256111, 282.93992876, 281.13785866, 274.18813219],
        [289.66798233, 275.61174478, 281.18572057, 287.43018937]],

       [[280.36889184, 285.40997739, 283.60869813, 279.24968147],
        [281.69619933, 290.08136561, 284.84447465, 285.59575017],
        [269.8330975 , 279.14079521, 293.55461826, 285.64462224]],

       [[281.13640875, 284.35807957, 276.55205582, 286.43130638],
        [285.30482789, 284.70838915, 288.35336769, 276.76259894],
        [281.67149162, 279.42301901, 277.06103343, 280.56601778]],

       [[281.88122358, 288.6934031 , 276.97437908, 278.71084183],
        [280.04093007, 284.2977853 , 282.94497753, 287.30868967],
        [277.87805624, 283.18829383, 279.29687476, 280.90954043]]])

## Section 1.2: Wrap the array: first attempt

Now we create a basic `DataArray` just by passing our plain `data` as input:

In [4]:
temp = xr.DataArray(data)
temp

<xarray.DataArray (dim_0: 5, dim_1: 3, dim_2: 4)>
array([[[284.29284765, 285.86800579, 278.37984255, 286.72043445],
        [283.99630883, 276.56693461, 277.55163841, 286.30030936],
        [281.17687724, 276.43815489, 283.39890798, 283.6687537 ]],

       [[284.04578007, 287.20918361, 283.45254156, 285.29731597],
        [296.50058016, 275.07994722, 288.28318583, 279.64042825],
        [272.43264769, 282.14561264, 289.45209141, 286.74879958]],

       [[283.35432589, 275.57918595, 282.99731907, 282.68861052],
        [279.20789625, 276.41291783, 281.22159553, 271.6345961 ],
        [275.4743125 , 285.98519659, 290.74406037, 280.60280354]],

       [[279.20846659, 283.21589759, 279.64400042, 289.12551152],
        [285.96380919, 283.71445668, 277.16387878, 284.58328429],
        [292.20874338, 289.86898128, 277.91991642, 295.00142191]],

       [[292.38646672, 281.61571524, 289.2068825 , 276.65959443],
        [288.66710414, 285.38720904, 284.95110921, 289.3252536 ],
        [289.6681178 , 279.65497949, 283.43240459, 282.84412755]]])
Dimensions without coordinates: dim_0, dim_1, dim_2

Note two things:

1. Xarray generates some basic dimension names for us (`dim_0`, `dim_1`, `dim_2`). We'll improve this with better names in the next example.
2. Wrapping the numpy array in a `DataArray` gives us a rich display in the notebook! (Try clicking the array symbol to expand or collapse the view)

## Section 1.3: Assign dimension names

Much of the power of Xarray comes from making use of named dimensions. So let's add some more useful names! We can do that by passing an ordered list of names using the keyword argument `dims`:

In [10]:
temp = xr.DataArray(data, dims=['time', 'lat', 'lon'])
temp

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[284.29284765, 285.86800579, 278.37984255, 286.72043445],
        [283.99630883, 276.56693461, 277.55163841, 286.30030936],
        [281.17687724, 276.43815489, 283.39890798, 283.6687537 ]],

       [[284.04578007, 287.20918361, 283.45254156, 285.29731597],
        [296.50058016, 275.07994722, 288.28318583, 279.64042825],
        [272.43264769, 282.14561264, 289.45209141, 286.74879958]],

       [[283.35432589, 275.57918595, 282.99731907, 282.68861052],
        [279.20789625, 276.41291783, 281.22159553, 271.6345961 ],
        [275.4743125 , 285.98519659, 290.74406037, 280.60280354]],

       [[279.20846659, 283.21589759, 279.64400042, 289.12551152],
        [285.96380919, 283.71445668, 277.16387878, 284.58328429],
        [292.20874338, 289.86898128, 277.91991642, 295.00142191]],

       [[292.38646672, 281.61571524, 289.2068825 , 276.65959443],
        [288.66710414, 285.38720904, 284.95110921, 289.3252536 ],
        [289.6681178 , 279.65497949, 283.43240459, 282.84412755]]])
Dimensions without coordinates: time, lat, lon

This has already been improved from a NumPy array, because we have names for each of the dimensions (or axes in NumPy parlance). Even better, we can associate arrays representing the values for the coordinates for each of these dimensions with the data when we create the `DataArray`. We'll see this in the next example.

# Section 2: Create a `DataArray` with named Coordinates

## Section 2.1: Make time and space coordinates

Here we will use [Pandas](https://foundations.projectpythia.org/core/pandas.html) to create an array of [datetime data](https://foundations.projectpythia.org/core/datetime.html), which we will then use to create a `DataArray` with a named coordinate `time`.

In [11]:
times = pd.date_range('2018-01-01', periods=5)
times

DatetimeIndex(['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04',
               '2018-01-05'],
              dtype='datetime64[ns]', freq='D')

We'll also create arrays to represent sample longitude and latitude:

In [12]:
lons = np.linspace(-120, -60, 4)
lats = np.linspace(25, 55, 3)

### Section 2.1.1: Initialize the `DataArray` with complete coordinate info

When we create the `DataArray` instance, we pass in the arrays we just created:

In [14]:
temp = xr.DataArray(data, coords=[times, lats, lons], dims=['time', 'lat', 'lon'])
temp

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[284.29284765, 285.86800579, 278.37984255, 286.72043445],
        [283.99630883, 276.56693461, 277.55163841, 286.30030936],
        [281.17687724, 276.43815489, 283.39890798, 283.6687537 ]],

       [[284.04578007, 287.20918361, 283.45254156, 285.29731597],
        [296.50058016, 275.07994722, 288.28318583, 279.64042825],
        [272.43264769, 282.14561264, 289.45209141, 286.74879958]],

       [[283.35432589, 275.57918595, 282.99731907, 282.68861052],
        [279.20789625, 276.41291783, 281.22159553, 271.6345961 ],
        [275.4743125 , 285.98519659, 290.74406037, 280.60280354]],

       [[279.20846659, 283.21589759, 279.64400042, 289.12551152],
        [285.96380919, 283.71445668, 277.16387878, 284.58328429],
        [292.20874338, 289.86898128, 277.91991642, 295.00142191]],

       [[292.38646672, 281.61571524, 289.2068825 , 276.65959443],
        [288.66710414, 285.38720904, 284.95110921, 289.3252536 ],
        [289.6681178 , 279.65497949, 283.43240459, 282.84412755]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0

### Section 2.1.2: Set useful attributes

We can also set some attribute metadata, which will help provide clear descriptions of the data. In this case, we can specify that we're looking at 'air_temperature' data and the units are 'kelvin'.

In [16]:
temp.attrs['units'] = 'kelvin'
temp.attrs['standard_name'] = 'air_temperature'

temp

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[284.29284765, 285.86800579, 278.37984255, 286.72043445],
        [283.99630883, 276.56693461, 277.55163841, 286.30030936],
        [281.17687724, 276.43815489, 283.39890798, 283.6687537 ]],

       [[284.04578007, 287.20918361, 283.45254156, 285.29731597],
        [296.50058016, 275.07994722, 288.28318583, 279.64042825],
        [272.43264769, 282.14561264, 289.45209141, 286.74879958]],

       [[283.35432589, 275.57918595, 282.99731907, 282.68861052],
        [279.20789625, 276.41291783, 281.22159553, 271.6345961 ],
        [275.4743125 , 285.98519659, 290.74406037, 280.60280354]],

       [[279.20846659, 283.21589759, 279.64400042, 289.12551152],
        [285.96380919, 283.71445668, 277.16387878, 284.58328429],
        [292.20874338, 289.86898128, 277.91991642, 295.00142191]],

       [[292.38646672, 281.61571524, 289.2068825 , 276.65959443],
        [288.66710414, 285.38720904, 284.95110921, 289.3252536 ],
        [289.6681178 , 279.65497949, 283.43240459, 282.84412755]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          kelvin
    standard_name:  air_temperature

### Section 2.1.3: Attributes are not preserved by default!

Notice what happens if we perform a mathematical operaton with the `DataArray`: the coordinate values persist, but the attributes are lost. This is done because it is very challenging to know if the attribute metadata is still correct or appropriate after arbitrary arithmetic operations.

To illustrate this, we'll do a simple unit conversion from Kelvin to Celsius:

In [18]:
temp_in_celsius = temp - 273.15
temp_in_celsius

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[11.14284765, 12.71800579,  5.22984255, 13.57043445],
        [10.84630883,  3.41693461,  4.40163841, 13.15030936],
        [ 8.02687724,  3.28815489, 10.24890798, 10.5187537 ]],

       [[10.89578007, 14.05918361, 10.30254156, 12.14731597],
        [23.35058016,  1.92994722, 15.13318583,  6.49042825],
        [-0.71735231,  8.99561264, 16.30209141, 13.59879958]],

       [[10.20432589,  2.42918595,  9.84731907,  9.53861052],
        [ 6.05789625,  3.26291783,  8.07159553, -1.5154039 ],
        [ 2.3243125 , 12.83519659, 17.59406037,  7.45280354]],

       [[ 6.05846659, 10.06589759,  6.49400042, 15.97551152],
        [12.81380919, 10.56445668,  4.01387878, 11.43328429],
        [19.05874338, 16.71898128,  4.76991642, 21.85142191]],

       [[19.23646672,  8.46571524, 16.0568825 ,  3.50959443],
        [15.51710414, 12.23720904, 11.80110921, 16.1752536 ],
        [16.5181178 ,  6.50497949, 10.28240459,  9.69412755]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0

For an in-depth discussion of how Xarray handles metadata, start in the Xarray docs [here](http://xarray.pydata.org/en/stable/getting-started-guide/faq.html#approach-to-metadata).

# Section 3: The `Dataset`: a container for `DataArray`s with shared coordinates

Along with `DataArray`, the other key object type in Xarray is the `Dataset`, which is a dictionary-like container that holds one or more `DataArray`s, which can also optionally share coordinates (see docs [here](http://xarray.pydata.org/en/stable/user-guide/data-structures.html#dataset)).

The most common way to create a `Dataset` object is to load data from a file (which we will practice in a later tutorial). Here, instead, we will create another `DataArray` and combine it with our `temp` data.

This will illustrate how the information about common coordinate axes is used.

## Section 3.1: Create a pressure `DataArray` using the same coordinates

This code mirrors how we created the `temp` object above.

In [23]:
pressure_data = 1000.0 + 5 * np.random.randn(5, 3, 4)
pressure = xr.DataArray(
    pressure_data, coords=[times, lats, lons], dims=['time', 'lat', 'lon']
)
pressure.attrs['units'] = 'hPa'
pressure.attrs['standard_name'] = 'air_pressure'

pressure

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[1002.70887468,  999.78532455,  995.53189213, 1002.86247191],
        [1014.79345442,  996.37632999,  997.41010098, 1001.05640211],
        [ 989.19948344, 1010.40208015, 1013.02000286,  999.68890331]],

       [[1003.7536842 , 1005.85791573,  996.26262725, 1000.92499235],
        [ 996.88306281,  995.39960579, 1003.32202689, 1002.92555376],
        [ 997.05468329,  999.37311705,  996.51479587,  997.11614868]],

       [[ 994.71470169,  996.60913562, 1004.5516367 , 1000.81796034],
        [1003.30563362, 1005.97323451,  997.83517168, 1008.91593154],
        [ 997.85115166, 1004.82538032, 1008.42982972, 1003.51053818]],

       [[ 999.23742148,  998.3627296 , 1001.64473157, 1006.2924927 ],
        [ 997.33252878,  996.70836856,  999.0988963 ,  998.23815245],
        [ 999.7353166 , 1000.37079419,  993.30975431,  997.05141267]],

       [[1009.96471706, 1000.18458085, 1006.50308664,  992.33712368],
        [ 994.6863456 , 1000.37114073, 1000.92392504,  992.45886663],
        [ 998.95096676, 1006.403742  ,  996.91820474, 1003.90882127]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          hPa
    standard_name:  air_pressure

## Section 3.2: Create a `Dataset` object

Each `DataArray` in our `Dataset` needs a name! 

The most straightforward way to create a `Dataset` with our `temp` and `pressure` arrays is to pass a dictionary using the keyword argument `data_vars`:

In [27]:
ds = xr.Dataset(data_vars={'Temperature': temp, 'Pressure': pressure})
ds

<xarray.Dataset>
Dimensions:      (time: 5, lat: 3, lon: 4)
Coordinates:
  * time         (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat          (lat) float64 25.0 40.0 55.0
  * lon          (lon) float64 -120.0 -100.0 -80.0 -60.0
Data variables:
    Temperature  (time, lat, lon) float64 284.3 285.9 278.4 ... 283.4 282.8
    Pressure     (time, lat, lon) float64 1.003e+03 999.8 ... 996.9 1.004e+03

Notice that the `Dataset` object `ds` is aware that both data arrays sit on the same coordinate axes.

## Section 3.3: Access Data variables and Coordinates in a `Dataset`

We can pull out any of the individual `DataArray` objects in a few different ways.

Using the "dot" notation:

In [28]:
ds.Pressure

<xarray.DataArray 'Pressure' (time: 5, lat: 3, lon: 4)>
array([[[1002.70887468,  999.78532455,  995.53189213, 1002.86247191],
        [1014.79345442,  996.37632999,  997.41010098, 1001.05640211],
        [ 989.19948344, 1010.40208015, 1013.02000286,  999.68890331]],

       [[1003.7536842 , 1005.85791573,  996.26262725, 1000.92499235],
        [ 996.88306281,  995.39960579, 1003.32202689, 1002.92555376],
        [ 997.05468329,  999.37311705,  996.51479587,  997.11614868]],

       [[ 994.71470169,  996.60913562, 1004.5516367 , 1000.81796034],
        [1003.30563362, 1005.97323451,  997.83517168, 1008.91593154],
        [ 997.85115166, 1004.82538032, 1008.42982972, 1003.51053818]],

       [[ 999.23742148,  998.3627296 , 1001.64473157, 1006.2924927 ],
        [ 997.33252878,  996.70836856,  999.0988963 ,  998.23815245],
        [ 999.7353166 , 1000.37079419,  993.30975431,  997.05141267]],

       [[1009.96471706, 1000.18458085, 1006.50308664,  992.33712368],
        [ 994.6863456 , 1000.37114073, 1000.92392504,  992.45886663],
        [ 998.95096676, 1006.403742  ,  996.91820474, 1003.90882127]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          hPa
    standard_name:  air_pressure

... or using dictionary access like this:

In [29]:
ds['Pressure']

<xarray.DataArray 'Pressure' (time: 5, lat: 3, lon: 4)>
array([[[1002.70887468,  999.78532455,  995.53189213, 1002.86247191],
        [1014.79345442,  996.37632999,  997.41010098, 1001.05640211],
        [ 989.19948344, 1010.40208015, 1013.02000286,  999.68890331]],

       [[1003.7536842 , 1005.85791573,  996.26262725, 1000.92499235],
        [ 996.88306281,  995.39960579, 1003.32202689, 1002.92555376],
        [ 997.05468329,  999.37311705,  996.51479587,  997.11614868]],

       [[ 994.71470169,  996.60913562, 1004.5516367 , 1000.81796034],
        [1003.30563362, 1005.97323451,  997.83517168, 1008.91593154],
        [ 997.85115166, 1004.82538032, 1008.42982972, 1003.51053818]],

       [[ 999.23742148,  998.3627296 , 1001.64473157, 1006.2924927 ],
        [ 997.33252878,  996.70836856,  999.0988963 ,  998.23815245],
        [ 999.7353166 , 1000.37079419,  993.30975431,  997.05141267]],

       [[1009.96471706, 1000.18458085, 1006.50308664,  992.33712368],
        [ 994.6863456 , 1000.37114073, 1000.92392504,  992.45886663],
        [ 998.95096676, 1006.403742  ,  996.91820474, 1003.90882127]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          hPa
    standard_name:  air_pressure

We'll return to the `Dataset` object when we start loading data from files in later tutorials today.

# Summary

In our initial tutorial, we utilized the `DataArray` and `Dataset` objects to handle and analyze spatial data. We focused on organizing extensive global climate datasets and examining the fluctuations within Earth's climate system.